In [112]:
import numpy as np
import pandas as pd
import os
from datetime import date
import sqlalchemy
import pymysql
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [163]:
file_name = '..\..\..\dataExport\LOGdata\log_2022_4_20.csv'
log_rawDF = pd.read_csv(file_name, sep='*')

In [164]:
log_rawDF.head(5)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN


In [165]:
feed_temp_ins4 = log_rawDF.query('Time >= 1618561800 and Time <= 1625228995 and Unit =="FPC14" and Parameter == "program"')

In [166]:
feed_temp_ins4.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
6,2021-4-20 13:3:58,1618916638,271269,3,FPC14,program,step ID - Changed (new and old value),4_WaterAcid,5_empty
7,2021-4-20 13:4:1,1618916641,101959,3,FPC14,program,start at - Changed (new and old value),sellected step at...,first step
8,2021-4-20 13:4:6,1618916646,386245,3,FPC14,program,procedure running - Changed,not active,active
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
10,2021-4-20 13:4:7,1618916647,642814,2,FPC14,program,Step 4_WaterAcid started,NaN,NaN


In [167]:
programID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('PrgID', regex=False, case=False, na=False)]

In [168]:
programID_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
1436,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
25922,2021-4-21 22:0:5,1619035205,342017,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
27652,2021-4-22 10:0:5,1619078405,339214,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
29425,2021-4-22 22:0:5,1619121605,341318,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
30376,2021-4-23 10:0:5,1619164805,351891,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [169]:
programID_DF['Message'].unique()

array(['program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 50'],
      dtype=object)

In [170]:
StepID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Step', regex=False, case=False, na=False)]

In [171]:
StepID_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
6,2021-4-20 13:3:58,1618916638,271269,3,FPC14,program,step ID - Changed (new and old value),4_WaterAcid,5_empty
10,2021-4-20 13:4:7,1618916647,642814,2,FPC14,program,Step 4_WaterAcid started,NaN,NaN
16,2021-4-20 13:4:25,1618916665,180599,2,FPC14,program,Step 4_WaterAcid started,NaN,NaN
146,2021-4-20 14:4:1,1618920241,169613,2,FPC14,program,Step 5_empty started,NaN,NaN
1438,2021-4-20 22:0:6,1618948806,648291,2,FPC14,program,Step 1_Harvest started,NaN,NaN


In [172]:
StepID_DF['Message'].unique()

array(['step ID - Changed (new and old value)',
       'Step 4_WaterAcid started', 'Step 5_empty started',
       'Step 1_Harvest started', 'Step 2_SiliciumHCl started',
       'Step 3_CompleteNutrients started',
       'Record ProgramID = F_ThalaPseu_HalfNutr  StepID = 1_Harvest saved (old and new record)EmptyReactorHarvestVolumeProductCIPFeedVolumeStopCirculationDuringFeedPneumaticDuringFeedPneumaticTimeDripTimeAerationAfterDripTimeDripCyclesOzoneDuringWaitwaitingTimeHarvestTriggerLastFeedStartedTriggerFeedTriggerStepTrigger',
       'Step 1_water started', 'Step 2_Oxonia 2% started',
       'Step step1 started', 'Step step2 started', 'Step step3 started'],
      dtype=object)

In [173]:
program_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Program', regex=False, case=False, na=False)]

In [174]:
program_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
12,2021-4-20 13:4:11,1618916651,125940,2,FPC14,program,_======program F_ThalaPseu_HalfNutr stopped before ended!!===========,NaN,NaN
15,2021-4-20 13:4:25,1618916665,177004,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
172,2021-4-20 14:5:54,1618920354,625413,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
1436,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [175]:
L = ['program','Program']
just_programDF = feed_temp_ins4[feed_temp_ins4['Message'].str.startswith(tuple(L))]

In [176]:
just_programDF['Message'].unique()

array(['Program F_ThalaPseu_HalfNutr started',
       'Program F_ThalaPseu_HalfNutr ended',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 50',
       'Program CleanOxonia_0.5% started',
       'Program waterSpoeling started', 'Program waterSpoeling ended',
       'Program Clean feed line base started',
       'Program Clean feed line base ended'], dtype=object)

In [177]:
#df[~df.C.str.contains("XYZ")]

only_feed = just_programDF[just_programDF["Message"].str.contains("CleanOxonia|waterSpoeling|line")==False]

In [178]:
only_feed['Message'].unique()

array(['Program F_ThalaPseu_HalfNutr started',
       'Program F_ThalaPseu_HalfNutr ended',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 50'],
      dtype=object)

In [179]:
only_feed.head(20)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
15,2021-4-20 13:4:25,1618916665,177004,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
172,2021-4-20 14:5:54,1618920354,625413,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
1436,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
1437,2021-4-20 22:0:6,1618948806,644921,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1746,2021-4-20 23:40:9,1618954809,644779,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
24188,2021-4-21 11:23:12,1618996992,630347,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
24625,2021-4-21 13:26:31,1619004391,861094,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
25922,2021-4-21 22:0:5,1619035205,342017,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
25923,2021-4-21 22:0:6,1619035206,860679,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN


In [107]:
'''
(only_feed
 .assign(col=only_feed.groupby('Message').cumcount().add(1))
 .pivot(index='Message', columns='col', values='Time')
 .add_prefix('Time_')
 .reset_index().rename_axis(columns=None)
)
'''

"\n(only_feed\n .assign(col=only_feed.groupby('Message').cumcount().add(1))\n .pivot(index='Message', columns='col', values='Time')\n .add_prefix('Time_')\n .reset_index().rename_axis(columns=None)\n)\n"

In [94]:
#feed_reactor_instance4 = log_rawDF.query('Type == 2 and Parameter == "Reactor" and Time >= 1618561800 and Time <= 1625228995 and Unit =="FPC14"')

In [113]:
# manual_ins4 = log_rawDF.query('Time >= 1618561800 and Time <= 1625228995 and Unit =="FPC14" and Type == 3')

In [125]:
# only_feed
'''
for index, row in only_feed.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg
    

for row in only_feed.itertuples():
    print(row.Type)
    print(row.Index)
'''



In [247]:
only_feed_newIndex = only_feed.reset_index()

In [248]:
only_feed_newIndex= only_feed_newIndex.drop('index',1)

In [249]:
only_feed_newIndex.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1,2021-4-20 13:4:25,1618916665,177004,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
2,2021-4-20 14:5:54,1618920354,625413,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
3,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
4,2021-4-20 22:0:6,1618948806,644921,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN


In [245]:
for idex, row in only_feed_newIndex.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg

    check_str = "ended"
    if "ended" in start_msg:
        next_le = only_feed_newIndex.loc[idex+1]['Message']
        if "Vol" not in next_le:
            line = pd.DataFrame({"Message": "Hellow"}, index=[idex])
            clean_feed = pd.concat([only_feed_newIndex.iloc[:idex-1], line, only_feed_newIndex.iloc[idex-1:]]).reset_index(drop=True)
    else:
        print("Not find")
    

In [243]:
for idex, row in only_feed_newIndex.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg

    check_str = "ended"
    if "ended" in start_msg:
        next_le = only_feed_newIndex.loc[idex+1]['Message']
        print(idex, next_le)
        if "Vol" not in next_le:
            line = pd.DataFrame({"Message": new_msg}, index=[idex+0.5])
            clean_feed_newIndex = only_feed_newIndex.append(line, ignore_index=False)
            clean_feed_newIndex = clean_feed_newIndex.sort_index().reset_index(drop=True)

2 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
5 Program F_ThalaPseu_HalfNutr started
7 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
10 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
13 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
16 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
19 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
22 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
25 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
28 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
31 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
34 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
37 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65
40 Program F_ThalaPseu_HalfNutr started
42 Program F_ThalaPseu_HalfNutr started
44 Program F_ThalaPseu_HalfNutr started
46 program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 6

In [246]:
clean_feed.head(50)

,index,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,0,2021-4-20 13:4:7,1.618917e+09,639887.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1,1,2021-4-20 13:4:25,1.618917e+09,177004.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
2,2,2021-4-20 14:5:54,1.618920e+09,625413.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
3,3,2021-4-20 22:0:5,1.618949e+09,125813.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
4,4,2021-4-20 22:0:6,1.618949e+09,644921.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
5,5,2021-4-20 23:40:9,1.618955e+09,644779.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
6,6,2021-4-21 11:23:12,1.618997e+09,630347.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
7,7,2021-4-21 13:26:31,1.619004e+09,861094.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
8,8,2021-4-21 22:0:5,1.619035e+09,342017.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
9,9,2021-4-21 22:0:6,1.619035e+09,860679.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
